# Corresponding video: DE Zoomcamp 5.3.1 - First Look at Spark/PySpark



In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
import requests
import os

url_dwnld ='https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/'
file_name = "fhvhv_tripdata_2021-01.csv.gz"
# if the file already exists, it will be skipped
if not os.path.exists(f"./{file_name}"):
    response = requests.get(url_dwnld + file_name)
    open(f"./{file_name}", "wb").write(response.content)
    print(f"Downloaded {file_name}")
else:
    print(f"File {file_name} already exists")



File fhvhv_tripdata_2021-01.csv.gz already exists


In [4]:
import gzip
import shutil
with gzip.open(f"./fhvhv_tripdata_2021-01.csv.gz", "rb") as f_in:
    with open(f"./fhvhv_tripdata_2021-01.csv", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [6]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [7]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [8]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [9]:
!powershell -command "Get-Content fhvhv_tripdata_2021-01.csv -TotalCount 1001 | Out-File -Encoding ASCII head.csv"


In [10]:
import pandas as pd

In [11]:
df_pandas = pd.read_csv('head.csv')

In [12]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [13]:
spark.createDataFrame(df_pandas).schema

c:\Users\jraldua-veuthey\.virtualenvs\data-eng-zc-gcp-aws-Y2KD2JE8\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
c:\Users\jraldua-veuthey\.virtualenvs\data-eng-zc-gcp-aws-Y2KD2JE8\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

Integer - 4 bytes
Long - 8 bytes

In [14]:
from pyspark.sql import types

In [15]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [16]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [17]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [18]:
df = df.repartition(24)

In [19]:
df.write.parquet('fhvhv/2021/01/', mode='overwrite')

# Starting here...Corresponding video: DE Zoomcamp 5.3.2 - Spark DataFrames

In [20]:
df = spark.read.parquet('fhvhv/2021/01/')

In [21]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [35]:
df1 = df.select('hvfhs_license_num')\
        .filter(df.hvfhs_license_num == 'HV0005')
df1.show()

+-----------------+
|hvfhs_license_num|
+-----------------+
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
|           HV0005|
+-----------------+
only showing top 20 rows



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [22]:
from pyspark.sql import functions as F

In [43]:
df\
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dropoff_date|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+
|           HV0005|              B02510|2021-01-03 10:26:53|2021-01-03 10:56:53|         179|          48|   null| 2021-01-03|  2021-01-03|
|           HV0005|              B02510|2021-01-01 17:45:58|2021-01-01 18:11:41|         228|         114|   null| 2021-01-01|  2021-01-01|
|           HV0005|              B02510|2021-01-01 14:17:55|2021-01-01 14:23:24|          26|          26|   null| 2021-01-01|  2021-01-01|
|           HV0003|              B02864|2021-01-01 20:09:27|2021-01-01 20:24:01|         159|          60|   null| 2021-01-01|  2021-01-01|
|           HV0003| 

In [44]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [45]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [47]:
crazy_stuff('B0288')

'a/120'

In [ ]:
F.udf

In [48]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [58]:
df \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .show()


Py4JJavaError: An error occurred while calling o261.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 1 times, most recent failure: Lost task 0.0 in stage 25.0 (TID 58) (5CD2373R4X.itk.local executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonfun

In [60]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .head(10)


[Row(pickup_datetime=datetime.datetime(2021, 1, 1, 20, 9, 27), dropoff_datetime=datetime.datetime(2021, 1, 1, 20, 24, 1), PULocationID=159, DOLocationID=60),
 Row(pickup_datetime=datetime.datetime(2021, 1, 3, 12, 18, 7), dropoff_datetime=datetime.datetime(2021, 1, 3, 12, 25, 59), PULocationID=226, DOLocationID=145),
 Row(pickup_datetime=datetime.datetime(2021, 1, 3, 11, 42, 37), dropoff_datetime=datetime.datetime(2021, 1, 3, 11, 47, 25), PULocationID=234, DOLocationID=224),
 Row(pickup_datetime=datetime.datetime(2021, 1, 1, 17, 40, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 17, 49, 27), PULocationID=82, DOLocationID=157),
 Row(pickup_datetime=datetime.datetime(2021, 1, 1, 6, 18, 8), dropoff_datetime=datetime.datetime(2021, 1, 1, 6, 40, 21), PULocationID=223, DOLocationID=198),
 Row(pickup_datetime=datetime.datetime(2021, 1, 1, 18, 15, 13), dropoff_datetime=datetime.datetime(2021, 1, 1, 18, 21, 26), PULocationID=246, DOLocationID=48),
 Row(pickup_datetime=datetime.datetime(2021

In [ ]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag

HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,

HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,

HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,

HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,

HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,

HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,

HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,

HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,

HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,

